# Point cloud alignment

This notebook contains the implementation of the Iterative Closest Points(ICP) algorithm, which is used to estimate point cloud alignment. The data was captured with an Intel RealSense camera.  
  
The rotations and translation first are converted to world coordinate system, then the ICP is applied for refining rotations and translations.

In [12]:
%matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pickle
import ge
import matplotlib.cm
from sklearn.neighbors import  KDTree
import open3d

Using matplotlib backend: TkAgg


In [13]:
"""
The two point clouds should be in the same coordinate system to be able to 
estimate the translations and rotations. It returns the translation and rotation
of the model point cloud.

The inputs are assumed to be shape Nx3.
"""
def ICP(
        source:np.array,
        model:np.array,
        max_err:float=0.5,
        max_iterations:int=20,
        sample_percentage:float=1.0
        )->np.array:
    err=1000
    iter=0
    while(err>max_err):
        #sample random points from the source pointscloud, and create KDTree from the model cloud
        sourceSamples=samplePoints(source,sample_percentage)
        treeModel=KDTree(model)
        #Get the nearest neighbour for each point in the source pointcloud from the model pointcloud
        indices=treeModel.query(sourceSamples,k=1,return_distance=False)
        modelSamples=np.take(model,indices,axis=0)
        modelSamples=np.reshape(modelSamples,(modelSamples.shape[0],modelSamples.shape[-1]))
        #Translate clouds by the means
        centroidM=np.mean(modelSamples,axis=0)
        centroidSource=np.mean(sourceSamples,axis=0)
        modelAligned=modelSamples-centroidM
        sourceAligned=sourceSamples-centroidSource
        #Use svd to get rotation and translation
        covarianceMatrix=modelAligned.T@sourceAligned
        #print(covarianceMatrix.shape)
        U,S,Vt=np.linalg.svd(covarianceMatrix)
        rotation=U@Vt
        translation=centroidM-rotation@centroidSource

        #Calculate error, and the change in the error
        err=error((rotation@sourceAligned.T).T,modelAligned)
        if iter%5==0:
            print("Iteration ",iter," with error ", err)
        iter+=1
        if iter>max_iterations:
            break
        #Transform the source point cloud
        source=(rotation@source.T).T+translation
    return rotation,translation

def samplePoints(array:np.array,sample_percentage):
    sampleIndices=np.random.choice(array.shape[0],int(sample_percentage*array.shape[0]),replace=False)
    samples=np.take(array,sampleIndices,axis=0)
    return samples

def error(source:np.array,model:np.array)->float:
    diffArray=source-model
    distanceArray=np.linalg.norm(diffArray,axis=1)
    mean=np.mean(distanceArray)
    return mean     

In [14]:
def toWorld(points,rotation,translation)->np.array:
    return (rotation.T@(points-translation).T).T

def toLocal(points, rotation,translation)->np.array:
    return (rotation@points.T).T+translation.T

def processPCKLPointCloud(filePath, maxError=0.001,maxIterations=20):
    # load .pckl data from the RealSense sensor
    with open( filePath, "rb") as fh:
        data = pickle.load( fh )
    # a single view is stored in data[i]
    Nview = len( data ) # number of views

    # colormap for storing each view with different color
    cmap = matplotlib.cm.get_cmap( 'jet')

    # open PLY output object
    g = ge.GePly( filePath[:-5] + '.ply' ) # change extension .pckl to .ply

    referenceCloud=data[0]["points"]
    rot=data[0]["extrinsics"][:3,:3]
    translation=data[0]["extrinsics"][:3,[3]]
    referenceCloud=toWorld(referenceCloud,rot,translation.T)
    C = cmap( 0 / ( Nview - 1 ) )[:3] # skip alpha
    g.points( referenceCloud.T, color=C )

    # iterate views
    for i in range(1,Nview):
        # 3D points from a single view - 3 x n numpy array
        print(i)
        X = data[i]["points"]
        # rotation matrix, 3x3 numpy array
        R = data[i]['extrinsics'][:3, :3]
        # translation vector, 3x1 numpy array
        t = data[i]['extrinsics'][:3, [3]]

        # transform points to world coordinate system
        worldPoints=toWorld(X,R,t.T)
        # run ICP algorithm
        rotation,translation=ICP(worldPoints,referenceCloud,maxError,maxIterations)
        rotatedPoints=(rotation@worldPoints.T).T+translation.T
        # color for this view
        C = cmap( i / ( Nview - 1 ) )[:3] # skip alpha
        
        # ply output
        g.points( rotatedPoints.T, color=C )
    g.close()

In [15]:
filePath="Data/processing3DData/2019-12-09_11-06-02_data.pckl"
# load .pckl data from the RealSense sensor
processPCKLPointCloud(filePath)

1


C:\Users\hgand\AppData\Local\Temp\ipykernel_27348\61542521.py:15: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = matplotlib.cm.get_cmap( 'jet')


Iteration  0  with error  0.01545718450199236
Iteration  5  with error  0.013420989190177424
Iteration  10  with error  0.012630676288754155
Iteration  15  with error  0.012395385356254436
Iteration  20  with error  0.012383170143083487
2
Iteration  0  with error  0.01694899042444533
Iteration  5  with error  0.014832859600617596
Iteration  10  with error  0.0141047773735963
Iteration  15  with error  0.0138889612222297
Iteration  20  with error  0.013829223523259956
3
Iteration  0  with error  0.02781500566348922
Iteration  5  with error  0.022311504732378425
Iteration  10  with error  0.021591235210923265
Iteration  15  with error  0.02103972990847426
Iteration  20  with error  0.020445870404461938


In [16]:
cloud=open3d.io.read_point_cloud("Data/processing3DData/2019-12-09_11-06-02_data.ply")
open3d.visualization.draw_geometries([cloud])

![image info](Data/AlignedPointClouds.PNG "Aligned pointcloud")

In [17]:
cloud=open3d.io.read_point_cloud("Data/processing3DData/2019-12-09_11-06-02_data-raw.ply")
open3d.visualization.draw_geometries([cloud])

![image info](Data/UnalignedPointClouds.PNG "Unaligned pointcloud")